- create a cleaning / preporcessing pipeline as a function
- split the data
- do eda
- find profanity



In [14]:
# importing libraries
#import advertools as adv
import demoji
import json
import pandas as pd
import re, nltk
import warnings

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords, wordnet

lemmatizer = WordNetLemmatizer()
stop_word = stopwords.words('english')
warnings.filterwarnings(action="ignore")


In [15]:
df = pd.read_csv('../data/raw/cyberbullying_tweets.csv')
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [16]:
df['cyberbullying_type'].unique()

array(['not_cyberbullying', 'gender', 'religion', 'other_cyberbullying',
       'age', 'ethnicity'], dtype=object)

In [17]:
df.shape

(47692, 2)

## data cleaning steps

- do eda on all the datzset 
- replace emojis with word
- keep hastags, but remove # 
- remove usernames 
- remove punctuation
- remove stop words and two letter words
- tokenize


- find profanity words using word2vec or not <>
- add more words to the list of profanity???


In [24]:
def super_clean(df):

    # Opening JSON file
    f = open('../data/external/profanity_list.json') 
    profanity_data = json.load(f)

    # extract emojis
    def extract_emoji(txt):
        emoji_txt = demoji.findall(txt)
        emoji_keys = emoji_txt.keys()
        emoji_values = emoji_txt.values()
        return  ' '.join(list(map(str, emoji_keys))), ' '.join(list(map(str, emoji_values)))

    # extract hashtags
    def hashtags(txt):
        txt = re.findall("#([a-zA-Z0-9_]{1,50})", txt)
        return ' '.join(list(map(str, txt)))
    
    # extract mentions
    def mentions(txt):
        txt = re.findall("@([a-zA-Z0-9_]{1,50})", txt)
        return ' '.join(list(map(str, txt)))

    # lookup profanity manually
    def find_profanity(text):
        profanity_set = set() 
        for word in text.split(): 
            if word in profanity_data:
                profanity_set.add(word)
        return ' '.join(list(map(str, profanity_set)))

    def get_wordnet_pos(tag):
        if tag.startswith('J'):
            return wordnet.ADJ
        elif tag.startswith('V'):
            return wordnet.VERB
        elif tag.startswith('N'):
            return wordnet.NOUN
        elif tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN
        
    df['profanity_list'] = df['tweet_text'].apply(lambda x: find_profanity(x))
    df['hashtags'] = df['tweet_text'].apply(lambda x: hashtags(x))
    df['mentions'] = df['tweet_text'].apply(lambda x: mentions(x))
    df['emoji_face'] =  df['tweet_text'].apply(lambda x: extract_emoji(x)[0])
    df['emoji_names'] =  df['tweet_text'].apply(lambda x:extract_emoji(x)[1])

    # remove stopwords
    df['clean_txt'] = df['tweet_text'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_word or len(word) > 2))
    df['clean_txt'] =  df['clean_txt'].apply(lambda x: re.sub("@[A-Za-z0-9_]+","", x))
    #remove hashtags
    df['clean_txt'] =  df['clean_txt'].apply(lambda x: re.sub("#[A-Za-z0-9_]+","", x))

    df['clean_txt_emoji'] =   df['clean_txt'] + " " + df['emoji_names'] # add emoji as text to tweet
    df['clean_txt_emoji'] =  df['clean_txt_emoji'].str.lower() # to lowe case
    df['clean_txt_emoji'] = df['clean_txt_emoji'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_word)) #remove stop words
    df['clean_txt_emoji'] = df['clean_txt_emoji'].apply(lambda x: re.sub(r"http\S+", "", x)) # remove links
    df['clean_txt_emoji'] = df['clean_txt_emoji'].str.replace("[^a-zA-Z#]", " ") # puntuation
    df['clean_txt_emoji'] = df['clean_txt_emoji'].apply(lambda x: ' '.join(word for word in x.split() if len(word) > 2))
    df['tokenized'] = df['clean_txt_emoji'].apply(word_tokenize) # tokenize
    df['pos_tags'] = df['tokenized'].apply(nltk.tag.pos_tag) # add pos tag
    df['wordnet_pos'] = df['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x]) # extract a new pos tag
    df['lemmatized'] = df['wordnet_pos'].apply(lambda x: [ lemmatizer.lemmatize(word, tag) for word, tag in x]) # lemmatize
   
    return df


In [25]:
# cleaning_time < 5 minutes
clean_df = super_clean(df)

### After cleaning 
- Lemmatizations

In [26]:
clean_df.sample(4)

,tweet_text,cyberbullying_type,profanity_list,hashtags,mentions,emoji_face,emoji_names,clean_txt,clean_txt_emoji,tokenized,pos_tags,wordnet_pos,lemmatized
8885,RT @1scrag1: I talk with a number of single Am...,gender,,,1scrag1,,,RT : I talk with number single American ladies...,talk number single american ladies can underst...,"[talk, number, single, american, ladies, can, ...","[(talk, NN), (number, NN), (single, JJ), (amer...","[(talk, n), (number, n), (single, a), (america...","[talk, number, single, american, lady, can, un..."
27229,@jonrosenberg just checking. :),other_cyberbullying,,,jonrosenberg,,,just checking. :),checking,[checking],"[(checking, VBG)]","[(checking, v)]",[check]
42190,"Excuse my language, but I’ve heard of this thi...",ethnicity,blacks,,,,,"Excuse language, but I’ve heard this thing cal...",excuse language heard thing called magical neg...,"[excuse, language, heard, thing, called, magic...","[(excuse, NN), (language, NN), (heard, VBD), (...","[(excuse, n), (language, n), (heard, v), (thin...","[excuse, language, hear, thing, call, magical,..."
22593,Please check how many Muslims carried out terr...,religion,terrorist,,,,,Please check how many Muslims carried out terr...,please check many muslims carried terrorist at...,"[please, check, many, muslims, carried, terror...","[(please, VB), (check, VB), (many, JJ), (musli...","[(please, v), (check, v), (many, a), (muslims,...","[please, check, many, muslim, carry, terrorist..."


In [27]:
clean_df.iloc[2640]

tweet_text            I have to poop but I'm the only one at my regi...
cyberbullying_type                                    not_cyberbullying
profanity_list                                                         
hashtags                                                            fml
mentions                                                               
emoji_face                                                             
emoji_names                                                            
clean_txt                    I have poop but I'm the only one register 
clean_txt_emoji                                       poop one register
tokenized                                         [poop, one, register]
pos_tags                        [(poop, NN), (one, CD), (register, NN)]
wordnet_pos                        [(poop, n), (one, n), (register, n)]
lemmatized                                        [poop, one, register]
Name: 2640, dtype: object

In [28]:
clean_df[clean_df["emoji_face"].str.len() != 0]

,tweet_text,cyberbullying_type,profanity_list,hashtags,mentions,emoji_face,emoji_names,clean_txt,clean_txt_emoji,tokenized,pos_tags,wordnet_pos,lemmatized
21,Kids Love😘❤ @ Mohamad Bin Zayed City مدينة محم...,not_cyberbullying,,,,❤ 😘,red heart face blowing a kiss,Kids Love😘❤ @ Mohamad Bin Zayed City مدينة محم...,kids love mohamad bin zayed city red heart fac...,"[kids, love, mohamad, bin, zayed, city, red, h...","[(kids, NNS), (love, VBP), (mohamad, JJ), (bin...","[(kids, n), (love, v), (mohamad, a), (bin, n),...","[kid, love, mohamad, bin, zayed, city, red, he..."
80,@iMrBarfield it starts Thursday. My classes ar...,not_cyberbullying,,,iMrBarfield,😌,relieved face,starts Thursday. My classes are canceled for ...,starts thursday classes canceled tomorrow tho ...,"[starts, thursday, classes, canceled, tomorrow...","[(starts, NNS), (thursday, NN), (classes, VBZ)...","[(starts, n), (thursday, n), (classes, v), (ca...","[start, thursday, class, cancel, tomorrow, tho..."
135,mmmm #MKR Forget Deconstruction @lisamromano @...,not_cyberbullying,,MKR,lisamromano garydlum mattsparks88,🍋,lemon,mmmm Forget Deconstruction THIS Lemon Tart...,mmmm forget deconstruction lemon tart mmmm lemon,"[mmmm, forget, deconstruction, lemon, tart, mm...","[(mmmm, NN), (forget, VB), (deconstruction, NN...","[(mmmm, n), (forget, v), (deconstruction, n), ...","[mmmm, forget, deconstruction, lemon, tart, mm..."
144,RT @mykitchenrules: Nawwww 😭😭😭 #MKR,not_cyberbullying,,MKR,mykitchenrules,😭,loudly crying face,RT : Nawwww 😭😭😭,nawwww loudly crying face,"[nawwww, loudly, crying, face]","[(nawwww, RB), (loudly, RB), (crying, VBG), (f...","[(nawwww, r), (loudly, r), (crying, v), (face,...","[nawwww, loudly, cry, face]"
151,@justinbieber Your How I Get Through Bullying ...,not_cyberbullying,,,justinbieber,♥,heart suit,Your How I Get Through Bullying And Tough Tim...,get bullying tough times thinking got makes se...,"[get, bullying, tough, times, thinking, got, m...","[(get, VB), (bullying, JJ), (tough, JJ), (time...","[(get, v), (bullying, a), (tough, a), (times, ...","[get, bullying, tough, time, think, get, make,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42964,RT @syeoga: Hella is from the bay....and major...,ethnicity,bitches,,syeoga,©,copyright,RT : Hella from the bay....and majority LA bit...,hella bay and majority bitches hate word hella...,"[hella, bay, and, majority, bitches, hate, wor...","[(hella, NN), (bay, NN), (and, CC), (majority,...","[(hella, n), (bay, n), (and, n), (majority, n)...","[hella, bay, and, majority, bitch, hate, word,..."
46427,"— @oibanai ije, ur one of my faves but u alr k...",ethnicity,,,oibanai,☺,smiling face,"— ije, ur one faves but u alr know that hehe ...",ije one faves alr know hehe also love seeing t...,"[ije, one, faves, alr, know, hehe, also, love,...","[(ije, VB), (one, CD), (faves, VBZ), (alr, RB)...","[(ije, v), (one, n), (faves, v), (alr, r), (kn...","[ije, one, faves, alr, know, hehe, also, love,..."
46879,@ksorbs 😂😆😂 #classic #coon... As pawpaw would ...,ethnicity,,classic coon momentsimissthatoldmanofhonor hil...,ksorbs,😆 🙊 😂,grinning squinting face speak-no-evil monkey f...,😂😆😂 ... As pawpaw would DO 🙊 YEP I went ther...,pawpaw would yep went there grinning squinting...,"[pawpaw, would, yep, went, there, grinning, sq...","[(pawpaw, NN), (would, MD), (yep, VB), (went, ...","[(pawpaw, n), (would, n), (yep, v), (went, v),...","[pawpaw, would, yep, go, there, grin, squint, ..."
47610,@Lilcruz2 you really tryna get me in trouble o...,ethnicity,,coon,Lilcruz2,😂,face with tears of joy,you really tryna get trouble twitter😂😂😂,really tryna get trouble twitter face tears joy,"[really, tryna, get, trouble, twitter, face, t...","[(really, RB), (tryna, JJ), (get, NN), (troubl...","[(really, r), (tryna, a), (get, n), (trouble, ...","[really, tryna, get, trouble, twitter, face, t..."


Source for lemmatixation and tokenization : https://www.holisticseo.digital/python-seo/nltk/lemmatize

In [29]:
def lemma(words):
    lemmatizer = WordNetLemmatizer()
    lemma_list =  [lemmatizer.lemmatize(token, "v") for token in word_tokenize(words)]
    sentence = " ".join(lemma_list)
    return sentence

In [30]:
clean_df.to_csv("../data/processed/clean_df.csv")

In [31]:
ss = "freaking work please I have many things to does doesnt"
#clean_df['clean_txt_lemma'] = clean_df['super_clean_txt'].apply(lambda x: lemma(x))
print(lemma(ss))
#clean_df.sample(5)

freak work please I have many things to do doesnt


In [32]:
"whole me being republican thing must reall"

'whole me being republican thing must reall'

In [33]:
#28775 - bad cleaning
#40953 - ""
#24091 - ""
# 46427 - abbrevaiton check ALR
clean_df.iloc[40953]

tweet_text            Dr..ur genuine and cleaver journelist , first ...
cyberbullying_type                                            ethnicity
profanity_list                                                         
hashtags                                                               
mentions                                                               
emoji_face                                                             
emoji_names                                                            
clean_txt             Dr..ur genuine and cleaver journelist , first ...
clean_txt_emoji       genuine cleaver journelist first time one spea...
tokenized             [genuine, cleaver, journelist, first, time, on...
pos_tags              [(genuine, JJ), (cleaver, NN), (journelist, NN...
wordnet_pos           [(genuine, a), (cleaver, n), (journelist, n), ...
lemmatized            [genuine, cleaver, journelist, first, time, on...
Name: 40953, dtype: object

In [37]:
df.head()

,tweet_text,cyberbullying_type,profanity_list,hashtags,mentions,emoji_face,emoji_names,clean_txt,clean_txt_emoji,tokenized,pos_tags,wordnet_pos,lemmatized
0,"In other words #katandandre, your food was cra...",not_cyberbullying,,katandandre mkr,,,,"In words #katandandre, food crapilicious! #mkr",words food crapilicious,"[words, food, crapilicious]","[(words, NNS), (food, NN), (crapilicious, JJ)]","[(words, n), (food, n), (crapilicious, a)]","[word, food, crapilicious]"
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,,aussietv MKR theblock ImACelebrityAU today sun...,,,,Why #aussietv white? #MKR #theblock #ImACelebr...,white,[white],"[(white, JJ)]","[(white, a)]",[white]
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,,,XochitlSuckkks,,,@XochitlSuckkks classy whore? Or red velvet cu...,classy whore red velvet cupcakes,"[classy, whore, red, velvet, cupcakes]","[(classy, NN), (whore, NN), (red, JJ), (velvet...","[(classy, n), (whore, n), (red, a), (velvet, n...","[classy, whore, red, velvet, cupcake]"
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,,,Jason_Gio,,,"@Jason_Gio meh. :P thanks heads up, concerned ...",meh thanks heads concerned another angry dude ...,"[meh, thanks, heads, concerned, another, angry...","[(meh, JJ), (thanks, NNS), (heads, NNS), (conc...","[(meh, a), (thanks, n), (heads, n), (concerned...","[meh, thanks, head, concern, another, angry, d..."
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,,,RudhoeEnglish,,,@RudhoeEnglish This ISIS account pretending Ku...,isis account pretending kurdish account like i...,"[isis, account, pretending, kurdish, account, ...","[(isis, NN), (account, NN), (pretending, VBG),...","[(isis, n), (account, n), (pretending, v), (ku...","[isi, account, pretend, kurdish, account, like..."


In [38]:
df.to_csv('train_data.csv')

#### Using a Steemer or a Lemma
##### to stem or to lemm? 
- Using both and comparing the result
- knowing that lemma will go to the root word :; eg, lemma for better is good
- Steemma for better is ? - beta?


According to Stackoverflow: https://stackoverflow.com/questions/1787110/what-is-the-difference-between-lemmatization-vs-stemming

Lemmatization and stemming are special cases of normalization. They identify a canonical representative for a set of related word forms.

 Stemming usually refers to a crude heuristic process that chops off the ends of words in the hope of achieving this goal correctly most of the time, and often includes the removal of derivational affixes. Lemmatization usually refers to doing things properly with the use of a vocabulary and morphological analysis of words, normally aiming to remove inflectional endings only and to return the base or dictionary form of a word, which is known as the lemma

 The real difference between stemming and lemmatization is threefold:

Stemming reduces word-forms to (pseudo)stems, whereas lemmatization reduces the word-forms to linguistically valid lemmas. This difference is apparent in languages with more complex morphology, but may be irrelevant for many IR applications;
Lemmatization deals only with inflectional variance, whereas stemming may also deal with derivational variance;
In terms of implementation, lemmatization is usually more sophisticated (especially for morphologically complex languages) and usually requires some sort of lexica. Satisfatory stemming, on the other hand, can be achieved with rather simple rule-based approaches.
Lemmatization may also be backed up by a part-of-speech tagger in order to disambiguate homonyms.

# 